In [1]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime
import time

In [2]:
sns.set()
pal= sns.hls_palette(10, h=0.5)
sns.set_palette(pal)
#Avoid display of scientific notation and show precision of 4 decimals:
pd.set_option('display.float_format', lambda x: '%4f' %x)

In [3]:
#filepath & loading csv
filename='C:\\Users\\Admin\\Downloads\\uber_nyc_data.csv'
df_uber = pd.read_csv(filename)

In [4]:
#Info about the data
df_uber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30925738 entries, 0 to 30925737
Data columns (total 6 columns):
id                 int64
origin_taz         object
destination_taz    object
pickup_datetime    object
trip_distance      float64
trip_duration      object
dtypes: float64(1), int64(1), object(4)
memory usage: 1.4+ GB


In [5]:
#A top look of the data
df_uber.head()

,id,origin_taz,destination_taz,pickup_datetime,trip_distance,trip_duration
0,252581,7C,6A,2014-09-01 09:00:00,4.250000,0:15:11
1,252582,7B,15,2014-09-01 18:00:00,10.170000,0:34:05
2,252583,11,2A,2014-09-01 17:00:00,4.020000,0:17:06
3,252584,3B,4A,2014-09-01 13:00:00,1.460000,0:06:32
4,252585,2A,10,2014-09-01 14:00:00,8.310000,0:26:17


In [6]:
#lenght of unique data in id column
len(df_uber.id.unique())

30925738

In [7]:
#No duplicate data or checking for missing data or error data
len(df_uber[df_uber.duplicated()==True])

0

In [8]:
#Showing total null values per column. Using of .sum() to just show which columns.
df_uber.isnull().sum()

id                       0
origin_taz               0
destination_taz    1273023
pickup_datetime          0
trip_distance           38
trip_duration           38
dtype: int64

In [9]:
arr1 = df_uber.origin_taz.unique()
arr1

array(['7C', '7B', '11', '3B', '2A', '5B', '10', '2B', '9', '6B', '15',
       '4C', '5A', '8', '14', '4A', '7A', '1', '16', '3A', '2C', '6A',
       '12', '13', '3C', '4B', '5C', '17'], dtype=object)

In [10]:
arr2 = df_uber.destination_taz.unique()
arr2

array(['6A', '15', '2A', '4A', '10', '4C', '7A', '3C', '5B', '14', '8',
       nan, '7C', '12', '2C', '1', '6B', '5C', '9', '3A', '2B', '11',
       '7B', '5A', '13', '4B', '18', '16', '3B', '17'], dtype=object)

In [11]:
set(arr2) - set(arr1)

{'18', nan}

In [12]:
#Check some of the data with missing destination. 
#One idea to fill missing values would be predict destination based on the other features.
df_uber[df_uber.destination_taz.isnull()].head()

,id,origin_taz,destination_taz,pickup_datetime,trip_distance,trip_duration
15,252596,6B,NaN,2014-09-01 17:00:00,11.670000,0:23:45
32,252613,9,NaN,2014-09-01 16:00:00,80.690000,1:59:37
63,252644,9,NaN,2014-09-01 18:00:00,23.070000,1:03:37
141,252722,9,NaN,2014-09-01 10:00:00,21.730000,0:41:12
148,252729,4B,NaN,2014-09-01 10:00:00,4.620000,0:14:55


In [13]:
#values missing for trip distance and duration, none of which are missing the destination.
df38 = df_uber[df_uber.trip_duration.isnull() & df_uber.trip_distance.isnull()]
df38.head()

,id,origin_taz,destination_taz,pickup_datetime,trip_distance,trip_duration
15155317,39535,2A,11,2015-04-25 12:00:00,nan,NaN
15245057,1009076,2A,2A,2015-04-26 01:00:00,nan,NaN
16519652,15028665,2A,7C,2015-04-29 21:00:00,nan,NaN
17148253,22250173,2A,2A,2015-04-12 02:00:00,nan,NaN
17297563,23716998,2C,11,2015-04-25 13:00:00,nan,NaN


In [14]:
#All cases for which trip_duration is null match the cases for which trip_distance is also null:
len(df_uber[(df_uber.trip_duration.isnull()==False) & (df_uber.trip_distance.isnull())])

0

In [15]:
#None of the pickup time data offer minutes or seconds precision
len(df_uber[df_uber.pickup_datetime.apply(lambda x: x[-5:] == '00:00') == False])

0

In [16]:
def dateParser(s):
    """
    Function that takes a string in the format yyyy-mm-dd hh:mm:ss, and
    returns the same as a datetime object.
    """
    return datetime.datetime(int(s[0:4]), int(s[5:7]), int(s[8:10]), int(s[11:13]))
#Other methods were tried, all slower (the slowest being parsing while reading file in Pandas):
#return datetime.strptime(s, "%Y-%m-%d %H:%M:%S.%f")
#return np.datetime64(s) equivalent to: np.array(df_uber.pickup_datetime[0]).astype(np.datetime64)
#return pandas.Timestamp(s, "%Y-%m-%d %H:%M:%S.%f", tz='utc' )uber 
#source: http://stackoverflow.com/questions/11136006/python-pandas-what-is-the-fastest-way-to-create-a-datetime-index

In [17]:
#time required to def function & calling the function & then time reuired for calling fuction minus to the time for def function
t0 = time.time()
df_uber['pu_date_hour'] = df_uber.pickup_datetime.apply(dateParser)
time.time() - t0

50.18468761444092

In [18]:
#Pu_date start & end date & then difference between end & start
beginning = df_uber.pu_date_hour.min()
end = df_uber.pu_date_hour.max()
print(beginning, end , end - beginning)

2014-09-01 00:00:00 2015-09-01 00:00:00 365 days 00:00:00


In [19]:
#Drop repeated column
df_uber
df_uber = df_uber.drop('pickup_datetime', axis=1)
df_uber

,id,origin_taz,destination_taz,trip_distance,trip_duration,pu_date_hour
0,252581,7C,6A,4.250000,0:15:11,2014-09-01 09:00:00
1,252582,7B,15,10.170000,0:34:05,2014-09-01 18:00:00
2,252583,11,2A,4.020000,0:17:06,2014-09-01 17:00:00
3,252584,3B,4A,1.460000,0:06:32,2014-09-01 13:00:00
4,252585,2A,10,8.310000,0:26:17,2014-09-01 14:00:00
...,...,...,...,...,...,...
30925733,30693431,15,15,6.860000,0:27:37,2015-09-01 00:00:00
30925734,30693447,5B,2A,3.440000,0:11:07,2015-09-01 00:00:00
30925735,30693461,4C,NaN,6.170000,0:19:10,2015-09-01 00:00:00
30925736,30693466,2A,5A,1.870000,0:07:54,2015-09-01 00:00:00


In [20]:
#Get all federal holidays in the period
from pandas.tseries.holiday import USFederalHolidayCalendar
#if return_name is True, then name is series value and date is the series index, otherwise date is series value.
holidays = USFederalHolidayCalendar().holidays(beginning, end, return_name = True)
holidays

2014-09-01                     Labor Day
2014-10-13                  Columbus Day
2014-11-11                  Veterans Day
2014-11-27                  Thanksgiving
2014-12-25                     Christmas
2015-01-01                 New Years Day
2015-01-19    Martin Luther King Jr. Day
2015-02-16                Presidents Day
2015-05-25                  Memorial Day
2015-07-03                      July 4th
dtype: object

In [21]:
#Extract the weekday for each holiday
holidays.index.map(lambda x: x.strftime('%a'))

Index(['Mon', 'Mon', 'Tue', 'Thu', 'Thu', 'Thu', 'Mon', 'Mon', 'Mon', 'Fri'], dtype='object')

In [22]:
#For completeness, let's add the day before and after the holidays (better: add day after if Thursday)
holidays_all = pd.concat([holidays, "Day After" + holidays.shift(1, 'D'), "Day Before" + holidays.shift(-1, 'D')])
holidays_all = holidays_all.sort_index()
holidays_all.head()

2014-08-31       Day BeforeLabor Day
2014-09-01                 Labor Day
2014-09-02        Day AfterLabor Day
2014-10-12    Day BeforeColumbus Day
2014-10-13              Columbus Day
dtype: object

In [23]:
#Create a column with the date, without the time. This will be useful later.
t0 = time.time()
df_uber['pu_date'] = pd.Series(map(lambda x: x.astype('datetime64[D]'), df_uber['pu_date_hour'].values))
#df_uber['pu_date'] = df_uber['pu_date_hour'].dt.date #this is way less memory effective and 11 sec slower to execute
time.time() - t0

61.094980239868164

In [24]:
#Get month and year from pick up timestamp
df_uber['year'] = df_uber['pu_date'].dt.year
df_uber['month'] = df_uber['pu_date'].dt.month

In [25]:
t0 = time.time()
#Get trip pick up day of the month
df_uber['day'] = df_uber['pu_date'].dt.day
time.time() - t0

2.389714479446411

In [26]:
t0 = time.time()
#Get trip pick up hour from timestamp
df_uber['hour'] = df_uber['pu_date'].dt.hour
time.time() - t0

2.589939832687378

In [27]:
t0 = time.time()
#Get trip pick up weekday from timestamp
df_uber['weekday'] = df_uber['pu_date'].dt.dayofweek
time.time() - t0

2.679989814758301

In [28]:
df_uber.weekday.value_counts()

5    5142832
4    4805002
3    4625511
6    4331195
2    4301537
1    4010434
0    3709227
Name: weekday, dtype: int64

In [29]:
df_uber.head()

,id,origin_taz,destination_taz,trip_distance,trip_duration,pu_date_hour,pu_date,year,month,day,hour,weekday
0,252581,7C,6A,4.250000,0:15:11,2014-09-01 09:00:00,2014-09-01,2014,9,1,0,0
1,252582,7B,15,10.170000,0:34:05,2014-09-01 18:00:00,2014-09-01,2014,9,1,0,0
2,252583,11,2A,4.020000,0:17:06,2014-09-01 17:00:00,2014-09-01,2014,9,1,0,0
3,252584,3B,4A,1.460000,0:06:32,2014-09-01 13:00:00,2014-09-01,2014,9,1,0,0
4,252585,2A,10,8.310000,0:26:17,2014-09-01 14:00:00,2014-09-01,2014,9,1,0,0


In [30]:
df_uber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30925738 entries, 0 to 30925737
Data columns (total 12 columns):
id                 int64
origin_taz         object
destination_taz    object
trip_distance      float64
trip_duration      object
pu_date_hour       datetime64[ns]
pu_date            datetime64[ns]
year               int64
month              int64
day                int64
hour               int64
weekday            int64
dtypes: datetime64[ns](2), float64(1), int64(6), object(3)
memory usage: 2.8+ GB


In [31]:
uniq_dur = df_uber[df_uber.trip_duration.isnull() == False].trip_duration.unique()
uniq_dur

array(['0:15:11', '0:34:05', '0:17:06', ..., '6:21:14', '7:53:17',
       '3:54:35'], dtype=object)

In [32]:
#Among unique duration strings, find how many represent >= 10h of duration:
long_duration = [] #>= 10 hours or 600 minutes
for item in uniq_dur:
    if len(item) != 7:
                long_duration.append(item)
#Long_duration
print(len(long_duration))

386


In [33]:
#Check for the most unusual strings for trip duration: some erroneous entries need to be addressed
for item in uniq_dur:
    if len(item) > 8:
        print(item)

172032:14:50
172032:39:03
136:25:12
172032:17:45
172032:19:27
119:53:35
120:49:05


In [34]:
def duration_to_minutes(s):
    """
    Function that takes a string with the hh:mm:ss format and
    returns the integer equivalent of the total time in minutes, 
    or zero for missing values in a Pandas dataframe.
    """
    if pd.isnull(s):
        val = 0 #note: this fills with 0 the 38 instances with null (missing) values
    else:
        hms = s.split(':')
        val = int(hms[0])*60 + int(hms[1]) + int(hms[2])/60.0
    return val

In [35]:
#Transform the trip duration string into trip duration in minutes:
t0 = time.time()
df_uber['duration_min'] = df_uber.trip_duration.apply(duration_to_minutes)
time.time() - t0

53.648221015930176

In [36]:
df_uber.head()

,id,origin_taz,destination_taz,trip_distance,trip_duration,pu_date_hour,pu_date,year,month,day,hour,weekday,duration_min
0,252581,7C,6A,4.250000,0:15:11,2014-09-01 09:00:00,2014-09-01,2014,9,1,0,0,15.183333
1,252582,7B,15,10.170000,0:34:05,2014-09-01 18:00:00,2014-09-01,2014,9,1,0,0,34.083333
2,252583,11,2A,4.020000,0:17:06,2014-09-01 17:00:00,2014-09-01,2014,9,1,0,0,17.100000
3,252584,3B,4A,1.460000,0:06:32,2014-09-01 13:00:00,2014-09-01,2014,9,1,0,0,6.533333
4,252585,2A,10,8.310000,0:26:17,2014-09-01 14:00:00,2014-09-01,2014,9,1,0,0,26.283333


In [37]:
#Get the mean distance and duration for each origin-destination pair
df_DistDur = df_uber.groupby(['origin_taz', 'destination_taz'])[['trip_distance', 'duration_min']].mean()

In [38]:
df_DistDur.head()

trip_distance  duration_min
origin_taz destination_taz                             
1          1                     1.421855     10.565513
           10                    3.650791     17.977578
           11                    5.093679     22.489925
           12                    8.887789     29.102406
           13                   11.583137     32.156759

In [39]:
#Replace 38 missing values with the average distance and duration for the respective origin-destination pair
for i in df38.index:
    orig = df_uber.loc[i, 'origin_taz']
    dest = df_uber.loc[i, 'destination_taz']
    df_uber.loc[i, 'trip_distance'] = df_DistDur.loc[orig, dest].trip_distance
    df_uber.loc[i, 'duration_min'] = df_DistDur.loc[orig, dest].duration_min

In [40]:
#Calculate average trip speed (mph) to help in understanding outliers (like trips with duration >10 h):
df_uber['trip_mph_avg'] = df_uber.trip_distance/(df_uber.duration_min/60.0)

In [41]:
#Check that trip_distance and duration_min have been replaced, and trip speed has been calculated:
df_uber.iloc[df38.index, :].head()

,id,origin_taz,destination_taz,trip_distance,trip_duration,pu_date_hour,pu_date,year,month,day,hour,weekday,duration_min,trip_mph_avg
15155317,39535,2A,11,3.936804,NaN,2015-04-25 12:00:00,2015-04-25,2015,4,25,0,5,19.732101,11.970759
15245057,1009076,2A,2A,1.358751,NaN,2015-04-26 01:00:00,2015-04-26,2015,4,26,0,6,10.919522,7.465989
16519652,15028665,2A,7C,9.039374,NaN,2015-04-29 21:00:00,2015-04-29,2015,4,29,0,2,30.204064,17.956604
17148253,22250173,2A,2A,1.358751,NaN,2015-04-12 02:00:00,2015-04-12,2015,4,12,0,6,10.919522,7.465989
17297563,23716998,2C,11,4.833061,NaN,2015-04-25 13:00:00,2015-04-25,2015,4,25,0,5,24.385143,11.891817


In [42]:
#Drop redundant trip_duration columns
df_uber = df_uber.drop('trip_duration', axis=1)
df_uber = df_uber.drop('pu_date_hour', axis=1)

In [43]:
#Source: http://uberestimate.com/prices/New-York-City/ for Uber X
base_fare = 2.55
per_minute = 0.35
per_mile = 1.75
min_fare = 8

In [44]:
#When not possible to work with Pandas dataframe directly (due to memory usage), go back to array!
#test = df_uber[['duration_min', 'trip_distance']].values

In [45]:
#def est_revenue(arr):
#   rev = base_fare + arr[0] * per_minute + arr[1] * per_mile
#   return rev if rev > min_fare else min_fare

In [46]:
#df_uber['est_revenue'] = pd.Series(map(lambda x: est_revenue(x), test))

In [47]:
#Using eval() is even more efficient for large datasets than resorting to Numpy! 
#Local variables must be followed by @. Only for arithmetic, cannot use if statement, for example.
df_uber['est_revenue'] = df_uber.eval('@base_fare + duration_min * @per_minute + trip_distance * @per_mile')
df_uber.loc[df_uber.est_revenue < 8, 'est_revenue'] = min_fare

In [48]:
df_uber.head()

,id,origin_taz,destination_taz,trip_distance,pu_date,year,month,day,hour,weekday,duration_min,trip_mph_avg,est_revenue
0,252581,7C,6A,4.250000,2014-09-01,2014,9,1,0,0,15.183333,16.794731,15.301667
1,252582,7B,15,10.170000,2014-09-01,2014,9,1,0,0,34.083333,17.903178,32.276667
2,252583,11,2A,4.020000,2014-09-01,2014,9,1,0,0,17.100000,14.105263,15.570000
3,252584,3B,4A,1.460000,2014-09-01,2014,9,1,0,0,6.533333,13.408163,8.000000
4,252585,2A,10,8.310000,2014-09-01,2014,9,1,0,0,26.283333,18.970197,26.291667


In [49]:
#Check for very long duration entries, and effect on revenue numbers:

#>=100h:
print(len(df_uber[(df_uber.duration_min >= 6000)]))
#Greater than 100h: 7 entries (erroneous): system error? fraud? Total revenue is relevant: $14,459,978 (2.4% of total)
#172032:14:50 #probable system error: distance < 10 miles
#172032:39:03 #probable system error: distance < 10 miles
#136:25:12 #short distance
#172032:17:45 #probable system error: distance < 10 miles
#172032:19:27 #probable system error: distance < 10 miles
#119:53:35 #long distance, incorrect destination?
#120:49:05 #long distance, this entry has missing destination

#>16h and <100h : 116 entries
df_uber[(df_uber.duration_min > 960) & (df_uber.duration_min > 6000)].est_revenue.sum()

7


14459977.979999999

In [50]:
#Were the trips with distance and duration equal to zero actually cancelled (didn't happen)?

print(len(df_uber[(df_uber.duration_min == 0) & (df_uber.trip_distance == 0)]))
#24866
#There is at least one case almost daily. The 24866 cases represent a revenue of $198,928, based on $8 minimum fare.

#Note that origin and destination are the same, except for 910 trips: did these trips in fact occur?
df_uber[(df_uber.duration_min == 0) & (df_uber.trip_distance == 0) & (df_uber.origin_taz != df_uber.destination_taz)].\
head()

#910

#There is generally a fee associated with trip cancellation, so unless these trips represent a system error or fraud,
#there was no loss of the minimum fare revenue.

24866


,id,origin_taz,destination_taz,trip_distance,pu_date,year,month,day,hour,weekday,duration_min,trip_mph_avg,est_revenue
21726,274307,4B,8,0.000000,2014-09-29,2014,9,29,0,0,0.000000,nan,8.000000
21847,274428,4C,2A,0.000000,2014-09-29,2014,9,29,0,0,0.000000,nan,8.000000
21909,274490,5C,6A,0.000000,2014-09-29,2014,9,29,0,0,0.000000,nan,8.000000
22179,274760,11,4A,0.000000,2014-09-29,2014,9,29,0,0,0.000000,nan,8.000000
22359,274940,3C,1,0.000000,2014-09-30,2014,9,30,0,1,0.000000,nan,8.000000


In [51]:
#Check cases with distance equal to zero but duration greater than zero:

print(len(df_uber[(df_uber.duration_min > 0) & (df_uber.trip_distance == 0)]))


#The median duration for trips with zero distance is 10 seconds (mean= 2.4 minutes), so most of these 85,515 cases 
#possibly represent trips that were cancelled right after they were registered.

#3873 trips of the 85,515 cases show pick up and drop off in different taxi zones 
#(median duration: 10 seconds, mean: 4.5 minutes).

#Let's check the small dataset (277) out of the 3873 cases with duration > 5 minutes, but distance equal to zero:
#Maybe these cases represent some error with registering the distance traveled?

df_uber[(df_uber.duration_min > 5) & (df_uber.trip_distance) & (df_uber.origin_taz != df_uber.destination_taz)].\
head()

85515


,id,origin_taz,destination_taz,trip_distance,pu_date,year,month,day,hour,weekday,duration_min,trip_mph_avg,est_revenue
0,252581,7C,6A,4.250000,2014-09-01,2014,9,1,0,0,15.183333,16.794731,15.301667
1,252582,7B,15,10.170000,2014-09-01,2014,9,1,0,0,34.083333,17.903178,32.276667
2,252583,11,2A,4.020000,2014-09-01,2014,9,1,0,0,17.100000,14.105263,15.570000
3,252584,3B,4A,1.460000,2014-09-01,2014,9,1,0,0,6.533333,13.408163,8.000000
4,252585,2A,10,8.310000,2014-09-01,2014,9,1,0,0,26.283333,18.970197,26.291667


In [52]:
#Check the trips with average speed slower than walking (3mph).
print(len(df_uber[(df_uber.trip_mph_avg <= 3)])) #262,666 cases

#Considering that some really bad traffic is possible, let's check the proportion of cases that fall under 
#2 miles or less traveled distance:
df_uber[(df_uber.trip_mph_avg <= 3) & (df_uber.trip_distance <= 2)].head(10) #246,225, or ~94% of all trips with < 3mph

#0.8% of cases from the entire dataset have calculated speed < 3mph : really bad traffic cases, interrupted trips, 
#or fraud? We will assume they're mostly really bad traffic cases or interrupted trips for this analysis.

262666


,id,origin_taz,destination_taz,trip_distance,pu_date,year,month,day,hour,weekday,duration_min,trip_mph_avg,est_revenue
82,252663,14,14,0.020000,2014-09-01,2014,9,1,0,0,1.333333,0.900000,8.000000
120,252701,2A,2A,1.510000,2014-09-01,2014,9,1,0,0,33.883333,2.673881,17.051667
271,252852,8,8,0.000000,2014-09-01,2014,9,1,0,0,0.500000,0.000000,8.000000
542,253123,4C,4C,0.050000,2014-09-02,2014,9,2,0,1,1.100000,2.727273,8.000000
554,253135,2A,2A,0.000000,2014-09-02,2014,9,2,0,1,2.683333,0.000000,8.000000
605,253186,2A,2A,0.050000,2014-09-02,2014,9,2,0,1,1.450000,2.068966,8.000000
839,253420,2A,2A,0.470000,2014-09-02,2014,9,2,0,1,10.583333,2.664567,8.000000
896,253477,5C,6B,0.990000,2014-09-02,2014,9,2,0,1,22.183333,2.677686,12.046667
945,253526,14,14,0.040000,2014-09-02,2014,9,2,0,1,1.400000,1.714286,8.000000
1260,253841,4A,4A,0.060000,2014-09-02,2014,9,2,0,1,1.350000,2.666667,8.000000


In [53]:
#Among the trips with very slow speed (< 3 mph), the most suspicious cases are perhaps those that show very long
#duration. Let's say that 1.5h would have been enough, in most situations, to get outside of some traffic gridlock, 
#then part of these cases should be investigated more in depth, especially because they represent a significant 
#share of the revenue ($850K).

print(len(df_uber[(df_uber.trip_mph_avg <= 3) & (df_uber.duration_min > 90) & (df_uber.duration_min <= 960)].head(10))) #8393 cases

#Due to the lack of specific domain knowledge about the system that has generated this data, and unavailability of 
#more detailed trip information, these cases will be disregarded as suspicious in this analysis. It's possible, 
#for example, that some of the cases represent a driver waiting for a passenger during a trip, even though this is 
#not a typical behaviour in ride sharing.

10


In [54]:
len(df_uber[(df_uber.month == 8) & (df_uber.year==2015)])

3368433

In [55]:
#A few data points represent the zeroth hour of Sep, 2015, extending the time period beyond exactly 365 days:
len(df_uber[df_uber.pu_date == datetime.date(2015, 9, 1)])  #1852 data points to be censored for convenience. 

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.
  


1852

In [56]:
#Create dataframe to be used for visualization with exactly 365 days of data, and max trip duration of 16h:
df_viz = df_uber[(df_uber.pu_date != datetime.date(2015, 9 , 1)) & (df_uber.duration_min <= 960)].copy() #1975 cases

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.
  


In [57]:
df_viz[['duration_min', 'trip_distance', 'trip_mph_avg', 'est_revenue']].describe()

,duration_min,trip_distance,trip_mph_avg,est_revenue
count,30923763.000000,30923763.000000,30898897.000000,30923763.000000
mean,20.992307,5.199632,13.507113,19.241870
std,16.084562,6.130756,11.756930,15.080891
min,0.000000,0.000000,0.000000,8.000000
25%,10.850000,1.690000,8.409836,9.573333
50%,17.233333,3.090000,11.448763,14.350833
75%,26.683333,6.230000,16.266667,23.030833
max,959.300000,568.570000,22548.000000,1225.858333


In [58]:
#Required by Plotly:
import matplotlib.mlab as mlab
import chart_studio.plotly as py

In [59]:
from matplotlib.ticker import FuncFormatter #Call formatter function to format tick values
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox) #Create image box
from matplotlib._png import read_png #Load png file
from matplotlib.patches import Ellipse #Draw ellipse

In [60]:
#Create functions to format tick number
def thousands_comma(x, pos):
    """
    Args are the value and tick position. 
    Returns number with thousands comma and no decimals.
    """
    return '{:,.0f}'.format(x) #this is the new syntax for formatting

In [61]:
def thousands_comma(x,pos):
    """
    Args are the value and tick position. 
    Returns number of thousands with one decimal, and K in lieu of 3 zeros.
    """
    return '{: 0f}{}'.format(x*1e-3, 'K')  #old syntax: '%1.0fK' % (x * 1e-3)

In [62]:
def millions_format(x, pos):
    """
    Args are the value and tick position. 
    Returns number of millions with one decimal, and M in lieu of 6 zeros.
    """
    return '{:.1f}{}'.format(x * 1e-6, 'M')

In [63]:
def millions_currency(x, pos):
    """
    Args are the value and tick position. 
    Returns number of millions with a $ sign, M in lieu of 6 zeros, and no decimals.
    """
    return '{}{:.0f}{}'.format('$', x* 1e-6, 'M')

In [64]:
#Define robust function to automatically add annotated labels on bar plots.
#Inspiration from http://composition.al/blog/2015/11/29/a-better-way-to-add-labels-to-bar-charts-with-matplotlib/

def annotate_labels(ax, labels_list, **kwargs):
    """
    Function to automatically add labels on bar charts.
    It takes a plot axis, an ordered list of labels, and text kwargs.
    """
    (y_bottom, y_top) = ax.get_ylim()
    y_height = y_top - y_bottom
    
    rects = ax.patches
    
    for rect, label in zip(rect, labels_list):
        height = rect.get_height()
        p_height = (height / y_height) # Fraction of axis height taken up by this rectangle
        label_position = height + (y_height * 0.01)
        
        # If we can fit the label above the column, do that;
        # otherwise, put it inside the column (or else, change ylim)
        #if p_height > 0.95:
        #    label_position = height - (y_height * 0.05)

        #else:
        #    label_position = height + (y_height * 0.01)
        
        ax.test(test.get_x() + rect.get_width()/2., label_position, label, kwargs)
    return None

In [65]:
#Create a plotting dataframe with counts (number of trips) grouped by day:
byDate = df_viz.groupby('pu_date')['id'].count() #365 complete entries
byDate.head()

pu_date
2014-09-01    31592
2014-09-02    41496
2014-09-03    46339
2014-09-04    53402
2014-09-05    59718
Name: id, dtype: int64

In [66]:
byDate['2014-10-28': '2014-11-05'] #an uptick in rides on 11/1, the day before the NYC marathon!

pu_date
2014-10-28     52084
2014-10-29     61082
2014-10-30     66369
2014-10-31     71787
2014-11-01    104450
2014-11-02     57289
2014-11-03     50491
2014-11-04     52395
2014-11-05     61518
Name: id, dtype: int64

In [67]:
#Create a plot with the total number of trips per day, highlighting some changepoints associated with major holidays 
#and other weather and touristic/cultural events.!ls Data/icons

In [68]:
#Create a plot with the total number of trips per day, highlighting some changepoints associated with major holidays 
#and other weather and touristic/cultural events.

fig = plt.figure()

ax = byDate.plot(figsize = (16,8), fontsize = 12, ylim = (10000, 170000), color = 'navy')

formatter = FuncFormatter(thousands_format)
ax.yaxis.set_axis_bgcolor('#F9F9F9')  #make background color a lighter gray to better contrast with the icon's images.

plt.title('Total Trips per Day with Annotation of Some Major Events and Holidays', fontsize = 20, color = 'navy')
plt.tick_params(labelsize = 14)
plt.xlabel('')

#Add icon to indicate snowstorm on the plot:
#NYC's mayor ordered the city under curfew, and public transportation system was shut down late on 1/26/15.

img1 = read_jpg('C:\\Users\\Admin\\Pictures\\Saved Pictures\\nature-windows-10-wallpaper-396866.jpg')
imagebox = OffsetImage(img1, zoom = 0.6)
xy = ['2015-01-27', 25000]

ab = Annotationbox(imagebox, xy, 
                  xybox = (22.,10.), #position from xy to lower left corner of box
                  xycords = 'data', #will use data coordinates for x and y
                  boxcoords = 'offset points', #same idea as textcoords, in points from lower left corner
                   pad = 0.1, frameon = False) #padding around the image
ax.add_artist(ab)

#Add icon to indicate Thanksgiving on the plot:
img2 = read_jpg('C:\\Users\\Admin\\Pictures\\Saved Pictures\\3d-nature-wallpaper-for-windows-7-hd-desktop-10-hd-wallpapers.jpg')
imagebox = offsetImage(img2, zoom = 0.6)
xy = ['2014-11-27', 4000]
ab = AnnotationBbox(imagebox, xy, xybox = (5., -5.), xycords= 'data', boxcoords = 'offset points', pad=0.1, frameon = False)
ax.add_artist(ab)

#Add icon to indicate Pride Week on the plot:
img3 = read_jpg('C:\\Users\\Admin\\Pictures\\Saved Pictures\\3d-nature-wallpaper-for-windows-7-hd-desktop-10-hd-wallpapers.jpg'')
imagebox = offsetImage(img3, zoom=0.6)
xy = ['2014-12-25', 40000]
a = AnnotationBbox(imagebox, xy, xybox = (10., -5.), xycords = 'data', boxcoords = 'offset points', pad = 0.1, frameon = False)
ax.add_artist(ab)

#Add icon to indicate Pride Week on the plot:
img4 = read_jpg('C:\\Users\\Admin\\Pictures\\Saved Pictures\\3d-nature-wallpaper-for-windows-7-hd-desktop-10-hd-wallpapers.jpg'')
imagebox = offsetImage(img4, zoom = 0.6)
xy = ['2015-05-25', 60000]
                
a = AnnotationBox(imagebox, xy, xybox (1., 5.), xycords = 'data', boxcoords = 'offset points', pad = 0.1, frameon = False)
ax.add_artist(ab)
                


SyntaxError: EOL while scanning string literal (<ipython-input-68-22850d0bf475>, line 37)